# <center> TensorFlow </center>

![](tflogo.png)

*<p style='text-align:right'>By TensorFlow - vectors combined, edited - Begoon [Apache License 2.0] via Wikimedia Commons</p>*

- one of the most popular tools for deep learning
- released in 2015
- "framework for numerical computation based on dataflow graphs." [2] [Hope et al. (2017)]
- tensors - multidimensional arrays
- built in C++, used through higer level languages (Python) and libraries (keras, tf-slim)

![](sqcube.png)

![](greyrgb.png)

- image tensor flows through the computational graph

### Computational graph:

![](compgraph.png)

- composed of nodes and edges

*<p style='text-align:right'>Computational Graph. By Sebastian Raschka via Github</p>*

### Advantages and Disadvantages (a few):

- open source and continously developed
- good visualization tools 
- scalability and portability
- steeper learning curve to become good at it 
- implies solid knowledge of math
- http://tensorflow.org

### Types of Operations:

- Add, Sub, Div, Mul, Log, Exp, etc. (element wise math ops.)
- on arrays and matrices: Constant, Shape, Split, Concat, Slice, MatMul, etc.
- Sigmoid, ReLU, Convolution2D, SoftMax, etc.
- and more

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
a = tf.constant('Hello')
b = tf.constant(' World!')
c = a+b

In [3]:
print(c)

Tensor("add:0", shape=(), dtype=string)


In [4]:
with tf.Session() as sess: 
    runop = sess.run(c)
    
print(runop)

b'Hello World!'


In [5]:
c = tf.add(a,b)

with tf.Session() as sess:
    runop = sess.run(c)
    
print(runop)

b'Hello World!'


In [6]:
mat_a = tf.constant(np.arange(1,12, dtype=np.int32), shape=[2,2,3])
mat_b = tf.constant(np.arange(12,24, dtype=np.int32), shape=[2,3,2])
mul_c = tf.matmul(mat_a, mat_b)

with tf.Session() as sess:
    runop = sess.run(mul_c)
    
print(runop)

[[[ 88  94]
  [214 229]]

 [[484 508]
  [642 674]]]


### A typical TensorFlow project:

- (1). constructing the graph: tf.constant, tf.placeholder, tf.variable, tf.nn.sigmoid, tf.matmul, etc.
- (2). executing the graph (within a session)
- use tensorboard for visualization.

### Placeholders:

- allow for reusability
- can be populated with different data every time you run the graph (feed_dict)

In [7]:
#with tf.Session() as sess:
#    sess.run(ops)
#    sess.close()

In [8]:
x = tf.placeholder(tf.float32, name='X', shape=(4,9))
w = tf.placeholder(tf.float32, name='W', shape=(9,1))
b = tf.fill((4,1), -1., name='bias')
y = tf.matmul(x,w)+b
s = tf.reduce_max(y)

In [9]:
x_data = np.random.randn(4,9)
w_data = np.random.randn(9,1)

with tf.Session() as sess:
    out_p = sess.run(s, feed_dict={x:x_data, w:w_data})
    
print('Output: {}'.format(out_p))

Output: 2.081198215484619


### Optimization

- purpose of a typical project: learn associations between features and labels.

### <center> $f(x_{i}) = w^{T}x_{i} + b$ </center> 
### <center> $y_{i} = f(x_{i}) + \varepsilon_{i}$ </center>

- **Loss Function**: how far the model is from the truth.
- train the model to minimize loss (adjusting the parameters).
- **MSE (Mean Squared Error)** - averages square distances (y_true, y_pred):

## <center> $L(y, \hat y) = \frac{1}{n}\ ∑_{i=1}^{n}(y_{i}-\hat y_{i})^{2}$ </center>

- **Cross Entropy** - similarity between distributions:

## <center> $H(p,q)=-∑_{x}p(x)logq(x)$ </center>

In [10]:
# X, Y, W, b

x = tf.placeholder(tf.float32, shape=[None, 3])
y_true = tf.placeholder(tf.float32, shape=None)
w = tf.Variable([[0,0,0]], dtype=tf.float32, name='W')
b = tf.Variable(0, dtype=tf.float32, name='b')

# Output

y_pred = tf.matmul(w, tf.transpose(x))+b

In [11]:
# MSE (Mean Squared Error)

loss = tf.reduce_mean(tf.square(y_true-y_pred))

# Cross Entropy

loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_true, logits=y_pred)
loss = tf.reduce_mean(loss)

### Linear Regression 

- Batch Gradient Descent: entire dataset.
- Stochastic Gradient Descent (SGD): mini-batches. 
- In TensorFlow:

<pre>optimizer = tf.train.GradientDescentOptimizer(learning_rate)</pre>
<pre>train = optimizer.minimize(loss)</pre>


In [17]:
import numpy as np

x_data = np.random.randn(2000,3)
w_real = [0.4, 0.6, 0.2]
b_real = -0.3 
noise = np.random.randn(1,2000)*0.1 
y_data = np.matmul(w_real, x_data.T)+b_real+noise

In [18]:
num_iters = 10 
g = tf.Graph()
wb = []

with g.as_default():
    x = tf.placeholder(tf.float32, shape=[None, 3])
    y_true = tf.placeholder(tf.float32, shape=None)
    
    with tf.name_scope('inference') as scope:
        w = tf.Variable([[0,0,0]], dtype=tf.float32, name='W')
        b = tf.Variable(0, dtype=tf.float32, name='b')
        y_pred = tf.matmul(w, tf.transpose(x))+b
        
    with tf.name_scope('loss') as scope:
        loss = tf.reduce_mean(tf.square(y_true-y_pred))
        
    with tf.name_scope('training') as scope:
        lr = 0.5
        optimizer = tf.train.GradientDescentOptimizer(lr)
        train = optimizer.minimize(loss)
        
    init = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        sess.run(init)
        for step in range(num_iters):
            sess.run(train, {x:x_data, y_true:y_data})
            if(step%5==0):
                print(step, sess.run([w,b]))
                wb.append(sess.run([w,b]))
                
        print(10, sess.run([w,b]))

0 [array([[ 0.40146095,  0.63455784,  0.21207747]], dtype=float32), -0.34306645]
5 [array([[ 0.39997593,  0.60164249,  0.19840008]], dtype=float32), -0.2986823]
10 [array([[ 0.39997596,  0.60164261,  0.19840011]], dtype=float32), -0.29868239]
